In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'data\data.csv')

In [3]:
df['z_score'] = (df['Value'] - df['Value'].mean()) / df['Value'].std()
df['z_score'].max()

12.823404336944517

In [4]:
threshold = 3
df['is_outlier_z'] = np.abs(df['z_score']) > threshold
df[df['is_outlier_z']]

,Datetime,Value,z_score,is_outlier_z
64,2023-03-03 16:00:00,291.0,6.831133,True
65,2023-03-03 17:00:00,310.0,7.375885,True
73,2023-03-04 01:00:00,359.0,8.780772,True
121,2023-03-06 01:00:00,416.0,10.415027,True
122,2023-03-06 02:00:00,500.0,12.823404,True
386,2023-03-17 02:00:00,215.0,4.652126,True
387,2023-03-17 03:00:00,306.0,7.261201,True
407,2023-03-17 23:00:00,307.0,7.289872,True
408,2023-03-18 00:00:00,311.0,7.404556,True


In [5]:
upper_bound = np.percentile(df['Value'], 95)
upper_bound

79.0

In [6]:
df['Value'] =  df['Value'].apply(lambda x: upper_bound if x > upper_bound else x)

In [7]:
df['z_score'] = (df['Value'] - df['Value'].mean()) / df['Value'].std()
df['z_score'].max()
threshold = 3
df['is_outlier_z'] = np.abs(df['z_score']) > threshold
df[df['is_outlier_z']]

,Datetime,Value,z_score,is_outlier_z


In [8]:
df = df.drop(['z_score', 'is_outlier_z'], axis=1)
df.head()

,Datetime,Value
0,2023-03-01 00:00:00,62.0
1,2023-03-01 01:00:00,64.0
2,2023-03-01 02:00:00,79.0
3,2023-03-01 03:00:00,79.0
4,2023-03-01 04:00:00,79.0


In [9]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [10]:
df.set_index('Datetime', inplace=True)
df.head()

,Value
Datetime,
2023-03-01 00:00:00,62.0
2023-03-01 01:00:00,64.0
2023-03-01 02:00:00,79.0
2023-03-01 03:00:00,79.0
2023-03-01 04:00:00,79.0


In [11]:
df['hour'] = df.index.hour + 1
df['day_of_week'] = df.index.dayofweek
df['is_weekend'] = df['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
df.head()

,Value,hour,day_of_week,is_weekend
Datetime,,,,
2023-03-01 00:00:00,62.0,1,2,0
2023-03-01 01:00:00,64.0,2,2,0
2023-03-01 02:00:00,79.0,3,2,0
2023-03-01 03:00:00,79.0,4,2,0
2023-03-01 04:00:00,79.0,5,2,0


In [12]:
lag_hours = 3
for i in range(1, lag_hours+1):
    df[f'lag_{i}'] = df['Value'].shift(i)
df.head()

,Value,hour,day_of_week,is_weekend,lag_1,lag_2,lag_3
Datetime,,,,,,,
2023-03-01 00:00:00,62.0,1,2,0,NaN,NaN,NaN
2023-03-01 01:00:00,64.0,2,2,0,62.0,NaN,NaN
2023-03-01 02:00:00,79.0,3,2,0,64.0,62.0,NaN
2023-03-01 03:00:00,79.0,4,2,0,79.0,64.0,62.0
2023-03-01 04:00:00,79.0,5,2,0,79.0,79.0,64.0


In [13]:
df['rolling_mean_3'] = df['Value'].rolling(window=3).mean() 
df['rolling_std_3'] = df['Value'].rolling(window=3).std()
df.head()

,Value,hour,day_of_week,is_weekend,lag_1,lag_2,lag_3,rolling_mean_3,rolling_std_3
Datetime,,,,,,,,,
2023-03-01 00:00:00,62.0,1,2,0,NaN,NaN,NaN,NaN,NaN
2023-03-01 01:00:00,64.0,2,2,0,62.0,NaN,NaN,NaN,NaN
2023-03-01 02:00:00,79.0,3,2,0,64.0,62.0,NaN,68.333333,9.291573
2023-03-01 03:00:00,79.0,4,2,0,79.0,64.0,62.0,74.000000,8.660254
2023-03-01 04:00:00,79.0,5,2,0,79.0,79.0,64.0,79.000000,0.000000


In [14]:
df = df.dropna(axis=0)
dataset = df.drop('day_of_week', axis=1)
dataset.head()

,Value,hour,is_weekend,lag_1,lag_2,lag_3,rolling_mean_3,rolling_std_3
Datetime,,,,,,,,
2023-03-01 03:00:00,79.0,4,0,79.0,64.0,62.0,74.000000,8.660254
2023-03-01 04:00:00,79.0,5,0,79.0,79.0,64.0,79.000000,0.000000
2023-03-01 05:00:00,72.0,6,0,79.0,79.0,79.0,76.666667,4.041452
2023-03-01 06:00:00,68.0,7,0,72.0,79.0,79.0,73.000000,5.567764
2023-03-01 07:00:00,67.0,8,0,68.0,72.0,79.0,69.000000,2.645751


In [15]:
split_ratio = 0.8
split_index = int(len(dataset) * split_ratio)

train_df = dataset.iloc[:split_index]
train_pollutants = train_df[['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3']]
train_df = train_df.drop(columns=['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3'], axis=1)


test_df = dataset.iloc[split_index:]
test_pollutants = test_df[['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3']]
test_df = test_df.drop(columns=['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3'], axis=1)


KeyError: "None of [Index(['PM2.5', 'PM10', 'NO2', 'SO2', 'CO', 'O3'], dtype='object')] are in the [columns]"

,PM2.5,PM10,NO2,SO2,CO,O3
Datetime,,,,,,
2023-03-01 03:00:00,33.552161,61.841399,47.732852,16.364456,1.321475,71.472801
2023-03-01 04:00:00,40.621110,58.295819,51.040441,17.135703,1.624703,54.608085
2023-03-01 05:00:00,41.081425,72.370991,68.991335,14.828186,1.156327,75.530553
2023-03-01 06:00:00,33.468814,55.214149,43.639874,17.818128,1.082760,85.158645
2023-03-01 07:00:00,28.734178,65.622890,35.378090,17.341548,1.078360,76.015710
...,...,...,...,...,...,...
2023-03-25 14:00:00,9.040753,28.520307,22.565208,8.268744,0.716847,43.455600
2023-03-25 15:00:00,17.011932,23.926224,17.824597,2.481554,0.238976,40.244416
2023-03-25 16:00:00,14.306488,36.663122,12.549052,5.967363,0.593935,43.104099


In [ ]:
train_df.shape, test_df.shape

((592, 14), (149, 14))

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.svm import SVR
import warnings
warnings.filterwarnings("ignore", message=".*does not have valid feature names.*")

In [ ]:
X = dataset.drop(columns=['Value'])  
y = dataset['Value'] 

train_size = int(len(dataset) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
models = {
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "XGBoost": xgb.XGBRegressor(n_estimators=100, random_state=42),
    "SVR": SVR(kernel='rbf')
}


In [ ]:
def recursive_forecast(model, X_last, n_steps, rolling_window=3):
    predictions = []
    
    for i in range(n_steps):
        rolling_mean = np.mean(X_last[-rolling_window:])
        rolling_std = np.std(X_last[-rolling_window:])
        
        is_weekend = 1 if (i % 7) >= 5 else 0
        hour = (X_last[-1] + 1) % 24
        
        X_last = np.roll(X_last, shift=-1)
        X_last[-1] = rolling_mean
        X_last = np.roll(X_last, shift=-1)
        X_last[-2] = rolling_std
        X_last = np.roll(X_last, shift=-1)
        X_last[-3] = is_weekend
        X_last = np.roll(X_last, shift=-1)
        X_last[-4] = hour
        
        next_pred = model.predict(X_last.reshape(1, -1))[0]
        predictions.append(next_pred)
        
    return np.array(predictions)

In [ ]:
train_df.to_csv('data/train.csv', index=False, header=True)
test_df.to_csv('data/test.csv', index=False, header=True)

In [16]:
train_df.shape

(592, 8)